In [1]:
from sys import version, version_info
assert version_info >= (3, 7), "Need python >= 3.7, you have:\n{}".format(version_info)

# %load_ext blackcellmagic
# Usage: %%black

from dataclasses import dataclass, field
from typing import List
from queue import Queue, Empty
from random import shuffle, choice

import ipywidgets as widgets
from IPython.display import display, clear_output
import asyncio
from time import sleep

# Enable widgets:
# !jupyter nbextension install --py widgetsnbextension --user
# !jupyter nbextension enable widgetsnbextension --user --py

In [2]:
@dataclass
class Pairs:
    japanese: str
    english: str
    

@dataclass
class Player:
    name: str
    vocab: List[Pairs] = field(repr=False)
#     known: List[Pairs] = field(default_factory=lambda :[], init=False)
        
    def __post_init__(self):
        shuffle(self.vocab)
        
        self.to_check = Queue()
        for i in self.vocab:
            self.to_check.put(i)
            
        self.score = {
            "known": 0,
            "failed": 0,
            "skipped": 0,
        }
        
    def update(self, category):
        self.score[category] += 1
       
    def pick_word(self):
        choice = self.to_check.get_nowait()
        return choice
    
    def show_score(self):
        for k in sorted(self.score.keys()):
            print("{:<10}: {:>3}/{:>3} [{:>4.1%}]".format(
                k, self.score[k], len(self.vocab), self.score[k] / len(self.vocab)))


In [13]:
_raw = []
with open("./previous_words.txt", "r") as f:
    for i in f.readlines():
        _raw.append(i[:-1])

pairs_previous = [
    Pairs(english=eng, japanese=jap)
    for (eng, jap) in [x.split(" - ", maxsplit=1) for x in _raw]
]

_raw = []
with open("./new_words.txt", "r") as f:
    for i in f.readlines():
        _raw.append(i[:-1])

pairs_new = [
    Pairs(english=eng, japanese=jap)
    for (eng, jap) in [x.split(" - ", maxsplit=1) for x in _raw]
]

In [14]:
%gui asyncio

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # TODO: learn how to use futures proprely
        try:
            future.result()
        except asyncio.InvalidStateError:
            future.set_result(0)
    widget.on_click(getvalue)
    return future

In [15]:
# NEW = True
NEW = False

if NEW:
    pairs = pairs_new
    vocab_size = len(pairs) * len(players)
else:
    pairs = pairs_previous
    vocab_size = 15 * len(players)

_players = ["Dan", "Romeo", "Paul"]
players = [Player(name=name, vocab=pairs) for name in _players]
shuffle(players)

flags = {"japanese": "🇯🇵", "english": "🇬🇧", "aussie": "🇦🇺"}

question = widgets.HTML(
    value="Questions here!",
)

list_players = widgets.Select(
    options=[p.name for p in players],
    value=players[0].name,
    disabled=True
)

progress = widgets.IntProgress(
    value=0,
    min=0,
    max=vocab_size,
    step=1,
    description="[  0 /{:>3}]".format(vocab_size),
    bar_style='',
    orientation='horizontal'
)

status = widgets.ToggleButtons(
    options=['Known ', 'Unkown ', 'Skipped '],
    disabled=False,
    button_style='',
    icons=["check-circle", "question-circle", "forward"],
)
_mapping = {"Known ": "known", "Unkown ": "failed", "Skipped ": "skipped"}

next_b = widgets.Button(
    description='Next!',
    button_style='',
    icon='check'
)

async def f():
    for i in range(vocab_size):
        player_idx = i % len(players)
        curr_player = players[player_idx]
        list_players.value = curr_player.name
        word = curr_player.pick_word()

        q, a = choice([
            ["japanese", "english"],
            ["english", "japanese"],
        ])
        f_q, f_a = q, a
        if curr_player.name == "Dan":
            # Need the right flag
            f_q = "aussie" if q == "english" else f_q
            f_a = "aussie" if a == "english" else f_a

        question.value = 'From <b>{}</b> {} to <b>{}</b> {}, translate:<p><b style="font-size: 32px">{}</b>'.format(
            q, flags[f_q], a, flags[f_a],
            getattr(word, q),
        )
        _ = await wait_for_change(next_b, 'value')
        next_b.disabled = True
        
        curr_player.update(_mapping[status.value])
        
        progress.value += 1
        progress.description = "[{:>3} /{:>3}]".format(progress.value, vocab_size)
        
        # Avoid quick double press
        sleep(0.5)
        next_b.disabled = False
        
    print("All done")
    
    for p in players:
        print("~"*80)
        print(f"Name: {p.name}")
        p.show_score()

asyncio.ensure_future(f())

display(progress)
display(question)
display(list_players)
display(status)
next_b

IntProgress(value=0, description='[  0 / 45]', max=45)

HTML(value='Questions here!')

Select(disabled=True, options=('Dan', 'Paul', 'Romeo'), value='Dan')

ToggleButtons(icons=('check-circle', 'question-circle', 'forward'), options=('Known ', 'Unkown ', 'Skipped '),…

Button(description='Next!', icon='check', style=ButtonStyle())

All done
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Name: Dan
failed    :   0/210 [0.0%]
known     :  15/210 [7.1%]
skipped   :   0/210 [0.0%]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Name: Paul
failed    :   1/210 [0.5%]
known     :  14/210 [6.7%]
skipped   :   0/210 [0.0%]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Name: Romeo
failed    :   2/210 [1.0%]
known     :  13/210 [6.2%]
skipped   :   0/210 [0.0%]


* Is brain fart usually used in past tense

* Make text bigger

* Kensa -> investigation

* Understand usage of to try

* Senmonka 

* Ask more questions when evaluating

# Suggestions for improvements

- [ ] Timer: 20s
    + Put pressure
    - Put pressure
    - We're fast enough usually
    + only for Dan :D
- [ ] Type the words
    + ensure full knownledge
    ? hiragana only
    - make it slow
    ? maybe only for english -> japanese
- [ ] Issue with successive repeatitions
- [x] Keep choosing direction randomly
- [ ] Answer in a sentence, from japanese to english
- [x] Have Tatsuya ask the questions
- [x] Add minimal timer no to press next too often

